# Detecção de texto em imagens com EAST detector

- Paper da técnica: https://arxiv.org/pdf/1704.03155v2.pdf

<a href="https://colab.research.google.com/github/fabiobento/ocr-sis-emb-2024-2/blob/main/3-OCR_Detecção_de_texto_com_EAST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
if IN_COLAB:
    # Baixar imagens para o laboratório.
    !wget https://github.com/fabiobento/ocr-sis-emb-2024-2/raw/refs/heads/main/imagens.zip
    !unzip -n -q imagens.zip

In [ ]:
try:
    import google.colab
    IN_COLAB = True
    print("Executando no Google Colab")
except ImportError:
    IN_COLAB = False
    print("Não executando no Google Colab")

# Importando as bibliotecas

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
if IN_COLAB:
    from google.colab.patches import cv2_imshow
%pip install imutils==0.5.4
from imutils.object_detection import non_max_suppression    

In [119]:
# Definir função para exibição de imagens
def imshow(imagem):
    if IN_COLAB:
        cv2_imshow(imagem)
    else:
        # Exibir a imagem com matplotlib
        if len(imagem.shape) == 2:  # Se a imagem tiver apenas 2 dimensões (níveis de cinza)
            plt.imshow(imagem, cmap='gray')
            plt.axis('off')  # Oculta os eixos
            plt.show()               
        else:
            plt.imshow(cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB))        
            plt.axis('off')  # Oculta os eixos
            plt.show()   

# Processando a imagem

In [120]:
if IN_COLAB:
  # Baixar arquivo do modelo
  %pip install gdown
  !mkdir -p ./Modelos
  !gdown --id 1-RbGz-8K7kC_Fve6J0eLtcRZQhmKS3UQ -O ./Modelos/frozen_east_text_detection.pb

In [121]:
detector = './Modelos/frozen_east_text_detection.pb'
largura, altura = 320, 320

In [122]:
imagem = './Imagens/caneca.jpg'
min_confianca = 0.9 #@param {type: 'number', min:0.0, max:1.0}

In [ ]:
img = cv2.imread(imagem)
imshow(img)

In [124]:
original = img.copy()

In [ ]:
img.shape

In [ ]:
H = img.shape[0]
W = img.shape[1]
print(H, W)

In [ ]:
proporcao_W = W / float(largura)
proporcao_H = H / float(altura)
print(proporcao_W, proporcao_H)

In [ ]:
img = cv2.resize(img, (largura, altura))
H = img.shape[0]
W = img.shape[1]
print(H, W)

In [ ]:
imshow(img)

# Carregando a rede neural

In [130]:
nomes_camadas = ['feature_fusion/Conv_7/Sigmoid', 'feature_fusion/concat_3']

In [131]:
rede_neural = cv2.dnn.readNet(detector)

In [ ]:
img.shape

In [133]:
blob = cv2.dnn.blobFromImage(img, 1.0, (W, H), swapRB = True, crop = False)

In [ ]:
blob.shape # batch_size

In [135]:
rede_neural.setInput(blob)
scores, geometry = rede_neural.forward(nomes_camadas)

In [ ]:
scores

In [ ]:
geometry

In [ ]:
geometry[0,0,0]

In [ ]:
scores.shape

In [ ]:
scores.shape[2:4]

In [141]:
linhas, colunas = scores.shape[2:4]

In [ ]:
print(linhas, colunas)

In [143]:
caixas = []
confiancas = []

# Decodificando os valores

- Explicação sobre os [valores de geometria](https://stackoverflow.com/questions/55583306/decoding-geometry-output-of-east-text-detection)

In [144]:
def dados_geometricos(geometry, y):
  xData0 = geometry[0, 0, y]
  xData1 = geometry[0, 1, y]
  xData2 = geometry[0, 2, y]
  xData3 = geometry[0, 3, y]
  data_angulos = geometry[0, 4, y]
  return data_angulos, xData0, xData1, xData2, xData3

In [145]:
def calculos_geometria(data_angulos, xData0, xData1, xData2, xData3):
  (offsetX, offsetY) = (x * 4.0, y * 4.0)
  angulo = data_angulos[x]
  cos = np.cos(angulo)
  sin = np.sin(angulo)
  h = xData0[x] + xData2[x]
  w = xData1[x] + xData3[x]

  fimX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
  fimY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))

  inicioX = int(fimX - w)
  inicioY = int(fimY - h)

  return inicioX, inicioY, fimX, fimY

In [ ]:
linhas, colunas

In [ ]:
scores[0, 0, 0]

In [148]:
for y in range(0, linhas):
  #print(y)
  data_scores = scores[0, 0, y]

  data_angulos, xData0, xData1, xData2, xData3 = dados_geometricos(geometry, y)

  for x in range(0, colunas):
    if data_scores[x] < min_confianca:
      continue

    inicioX, inicioY, fimX, fimY = calculos_geometria(data_angulos, xData0, xData1, xData2, xData3)
    confiancas.append(data_scores[x])
    caixas.append((inicioX, inicioY, fimX, fimY))

In [ ]:
confiancas

In [ ]:
caixas

In [151]:
deteccoes = non_max_suppression(np.array(caixas), probs=confiancas)

In [ ]:
deteccoes

In [ ]:
proporcao_H, proporcao_W

In [ ]:
copia = original.copy()
for (inicioX, inicioY, fimX, fimY) in deteccoes:
  #print(inicioX, inicioY, fimX, fimY)
  inicioX = int(inicioX * proporcao_W)
  inicioY = int(inicioY * proporcao_H)
  fimX = int(fimX * proporcao_W)
  fimY = int(fimY * proporcao_H)

  # region of interest
  roi = copia[inicioY:fimY, inicioX:fimX]

  cv2.rectangle(original, (inicioX, inicioY), (fimX, fimY), (0,255,0), 2)
imshow(original)

In [ ]:
imshow(roi)

In [156]:
roi = cv2.resize(roi, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_CUBIC)

In [ ]:
imshow(roi)

# Reconhecimento do texto

In [ ]:
!sudo apt install tesseract-ocr
%pip install pytesseract

import pytesseract

Instalar pacotes de idiomas

In [ ]:
!mkdir tessdata
!wget -O ./tessdata/por.traineddata https://github.com/tesseract-ocr/tessdata/blob/main/por.traineddata?raw=true

In [ ]:
!tesseract --help-psm

In [161]:
config_tesseract = "--tessdata-dir tessdata --psm 7"

In [ ]:
copia = original.copy()
for (inicioX, inicioY, fimX, fimY) in deteccoes:
  inicioX = int(inicioX * proporcao_W)
  inicioY = int(inicioY * proporcao_H)
  fimX = int(fimX * proporcao_W)
  fimY = int(fimY * proporcao_H)

  roi = copia[inicioY:fimY, inicioX:fimX]
  texto = pytesseract.image_to_string(roi, lang='por', config=config_tesseract)
  print(texto)

  cv2.rectangle(original, (inicioX, inicioY), (fimX, fimY), (0,255,0), 2)
imshow(original)

## Expandindo o ROI do texto

In [ ]:
margem = 3
copia = original.copy()
for (inicioX, inicioY, fimX, fimY) in deteccoes:
  inicioX = int(inicioX * proporcao_W)
  inicioY = int(inicioY * proporcao_H)
  fimX = int(fimX * proporcao_W)
  fimY = int(fimY * proporcao_H)

  roi = copia[inicioY - margem:fimY + margem, inicioX - margem:fimX + margem]
  imshow(roi)
  texto = pytesseract.image_to_string(roi, lang='por', config=config_tesseract)
  print(texto)

  cv2.rectangle(copia, (inicioX - margem, inicioY - margem), (fimX + margem, fimY + margem), (0,255,0), 2)
imshow(copia)